In [2]:
import numpy as np
from numpy import array
import pandas as pd
import csv
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

In [106]:
PATH = 'hdfs_train.csv'
with open(PATH, newline='') as f:
    reader = csv.reader(f)
    data = list(reader)
for i in range(len(data)):
    while('' in data[i]):
        data[i].remove('')


In [108]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

trial = data[0]
res = [eval(i) for i in trial]
t = [int(i) for i in res]
t.pop(0)

# choose a number of time steps
n_steps = 5
# split into samples
X, y = split_sequence(t, n_steps)
# summarize the data
tot = [X,y]
for i in range(len(X)):
    print(X[i], y[i])

[ 5  5  5 22 11] 9
[ 5  5 22 11  9] 11
[ 5 22 11  9 11] 9
[22 11  9 11  9] 11
[11  9 11  9 11] 9
[ 9 11  9 11  9] 26
[11  9 11  9 26] 26
[ 9 11  9 26 26] 26
[11  9 26 26 26] 23
[ 9 26 26 26 23] 23
[26 26 26 23 23] 23
[26 26 23 23 23] 21
[26 23 23 23 21] 21
[23 23 23 21 21] 21


In [122]:
X ,Y = list(), list()
for j in range(100):
    temp = data[j]
    res = [eval(i) for i in temp]
    t = [int(i) for i in res]
    t.pop(0)
    # choose a number of time steps
    n_steps = 5
    # split into samples
    x, y = split_sequence(t, n_steps)
    # summarize the data
    for i in range(len(x)):
        X.insert(i,x[i])
        Y.insert(i,y[i])
X = array(X)
Y = array(Y)
print(len(X))

1265


In [124]:
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
model.add(LSTM(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, Y, epochs=500, verbose=0)
# demonstrate prediction
x_input = array([22,11,9,11,9])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

KeyboardInterrupt: ignored